# Creating a Cutout with ECMWF ensemble forecast

In this example we download ECMWF ensemble forecast **open-data** for a cutout we want to create.

This only works if you have in before

* Installed the ECMWF `earthkit` package
* Documentation on how to use `earthkit` to download data [on their website here](https://earthkit-data.readthedocs.io/en/latest/examples/ecmwf_open_data.html)

Open-data only contains a subset of forecast variables.
As a result, there are missing features in this cutout which we replace with the recent 3-year mean from ERA5.

In [ ]:
import gc
import pandas as pd
import earthkit.data as ekd
import dask.array as da
import xarray as xr


step_schedule = list(range(0, 366, 6))
model_config = dict(
    name="ecmwf-open-data",
    stream="enfo",
    type="pf",
    model="ifs",
    source="ecmwf",
)


def formate_forecast_to_analysis(
    forecast: xr.Dataset
):
    assert "longitude" in forecast.coords, "Dataset must have a longitude coordinate"
    assert forecast.longitude.min() >= -180 and forecast.longitude.max() <= 180, "Longitude values must be in the range [-180, 180]"
    return (
        # convert longitudes from [-180, 180] to [0, 360]
        forecast.assign_coords(longitude=(((forecast.longitude + 360) % 360)))
    )

def formate_analysis_to_forecast(
    analysis: xr.Dataset
):
    assert "longitude" in analysis.coords, "Dataset must have a longitude coordinate"
    assert analysis.longitude.min() >= 0 and analysis.longitude.max() <= 360, "Longitude values must be in the range [0, 360]"
    return (
        # 1. convert longitudes from [0, 360] to [-180, 180]
        analysis.assign_coords(longitude=(((analysis.longitude + 180) % 360) - 180))
    )
    

In [ ]:
forecast_date = "2025-09-26"
forecast_time = 0
steps = [18, 24]

In [ ]:
forecast_datetime = pd.to_datetime(forecast_date) + pd.Timedelta(hours=forecast_time)
reference_datetimes = [forecast_datetime + pd.Timedelta(hours=s) for s in steps]
assert all(s in step_schedule for s in steps), f"Step {steps} not in schedule {step_schedule}"

## Height

Geopotential: z

In [ ]:
ds = ekd.from_source(
    "ecmwf-open-data",
    date=forecast_date,
    time=forecast_time,
    param="z",
    levtype="sfc",
    step=0,
    model="ifs",
    source="ecmwf",
).to_xarray()

def _add_height(ds):
    g0 = 9.80665
    z = ds["z"]
    return z / g0

height = _add_height(ds)

In [ ]:
height

## Wind

In [ ]:
ds = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param=["10u", "10v", "100u", "100v"],
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()

In [ ]:
wind_spd = {}
for h in [10, 100]:
    w = da.sqrt(ds[f"{h}u"]**2 + ds[f"{h}v"]**2)
    w = w.assign_attrs(
            units=ds[f"{h}u"].attrs["units"], long_name=f"{h} metre wind speed"
        )
    wind_spd[h] = w

In [ ]:
wind_spd[100]

In [ ]:
wnd_shear_exp = (
    da.log(wind_spd[10] / wind_spd[100]) / da.log(10 / 100)
).assign_attrs(units="", long_name="wind shear exponent")
wnd_shear_exp

In [ ]:
azimuth = da.arctan2(ds["100u"], ds["100v"])
wnd_azimuth = azimuth.where(azimuth >= 0, azimuth + 2 * da.pi)
wnd_azimuth

In [ ]:
del ds
gc.collect()

`fsr` (Forecast surface roughness) is not available in `ecmwf-opendata` so we will specify `extrapolate_wind_speed` with `method='power'` to avoid the usage of `fsr`

## Influx

In [ ]:
ds = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param=["ssrd", "ssr"],
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()
ds

Again, since `ecmwf-opendata` does not contain `fdir` and `tisr`, we use the 3-year average from ERA5.

In [ ]:
import cdsapi

missing_influx_vars = {
    "tisr": "toa_incident_solar_radiation",
    "fdir": "total_sky_direct_solar_radiation_at_surface",
}

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": list(missing_influx_vars.values()),
    "year": list(set([str(ref_dt.year - i) for ref_dt in reference_datetimes for i in range(1, 4)])),
    "month": list(set([str(ref_dt.month) for ref_dt in reference_datetimes])),
    "day": list(set([str(ref_dt.day) for ref_dt in reference_datetimes])),
    "time": [f"{ref_dt.hour:02d}:00" for ref_dt in reference_datetimes],
    "data_format": "grib",
    "download_format": "unarchived"
}

client = cdsapi.Client()
result = client.retrieve(dataset, request)
result.download("climo_influx.grib")

In [ ]:
import xarray as xr

ds_influx = (
    xr.open_dataset("climo_influx.grib", decode_timedelta=True, engine="cfgrib")
    .rename({"fdir": "influx_direct", "tisr": "influx_toa"})
)
ds_influx = (
    ds_influx
    # convert longitudes from [0, 360] to [-180, 180]
    .assign_coords(longitude=(((ds_influx.longitude + 180) % 360) - 180))
    .sortby("longitude")
)
ds_influx

In [ ]:
climo_influx = (
    ds_influx
    .assign_coords(
        month=ds_influx["valid_time"].dt.month,
        day=ds_influx["valid_time"].dt.day,
        hour=ds_influx["valid_time"].dt.hour,
    )
    .groupby(["month", "day", "hour"])
    .mean()
    .drop_vars("number")
)
climo_influx

In [23]:
ds_forecast = xr.concat(
    [
        (
            climo_influx.sel(
                month=rd.month,
                day=rd.day,
                hour=rd.hour,
            )
            .assign_coords(step=pd.Timedelta(hours=s))
            .drop_vars(["month", "day", "hour"])
        )
        for rd, s in zip(reference_datetimes, steps)
    ],
    dim="step"
)
ds_forecast

<xarray.Dataset> Size: 17MB
Dimensions:        (step: 2, latitude: 721, longitude: 1440)
Coordinates:
    surface        float64 8B 0.0
  * latitude       (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * step           (step) timedelta64[ns] 16B 18:00:00 1 days
Data variables:
    influx_toa     (step, latitude, longitude) float32 8MB 0.0 0.0 ... 1.409e+05
    influx_direct  (step, latitude, longitude) float32 8MB 0.0 0.0 ... 1.958e+04
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-28T13:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [24]:
ds_forecast["albedo"] = (
    ((ds["ssrd"] - ds["ssr"]) / ds["ssrd"].where(ds["ssrd"] != 0))
    .fillna(0.0)
    .assign_attrs(units="(0 - 1)", long_name="Albedo")
)

ds_forecast["influx_diffuse"] = (
    (ds["ssrd"] - ds_forecast["influx_direct"])
    .assign_attrs(
        units="J m**-2", long_name="Surface diffuse solar radiation downwards"
    )
)

# Convert from energy to power J m**-2 -> W m**-2 and clip negative fluxes
for a in ("influx_direct", "influx_diffuse", "influx_toa"):
    ds_forecast[a] = ds_forecast[a] / (60.0 * 60.0)
    ds_forecast[a].attrs["units"] = "W m**-2"

ds_forecast

<xarray.Dataset> Size: 2GB
Dimensions:         (step: 2, latitude: 721, longitude: 1440, number: 50)
Coordinates:
    surface         float64 8B 0.0
  * latitude        (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude       (longitude) float64 12kB -180.0 -179.8 ... 179.5 179.8
  * step            (step) timedelta64[ns] 16B 18:00:00 1 days
  * number          (number) int64 400B 1 2 3 4 5 6 7 8 ... 44 45 46 47 48 49 50
Data variables:
    influx_toa      (step, latitude, longitude) float32 8MB 0.0 0.0 ... 39.13
    influx_direct   (step, latitude, longitude) float32 8MB 0.0 0.0 ... 5.44
    albedo          (number, step, latitude, longitude) float64 831MB 0.0 ......
    influx_diffuse  (number, step, latitude, longitude) float64 831MB 0.0 ......
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-28T13:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
# ds_forecast = xr.concat(
#     [
#         ds_forecast
#         .assign_coords(
#             time=forecast_datetime,
#         )
#     ],
#     dim="time"
# )
# ds_forecast = (
#     ds_forecast
#     # convert longitudes from [-180, 180] to [0, 360]
#     .assign_coords(longitude=(((ds_forecast.longitude + 360) % 360)))
#     # rename the longitude and latitude coordinates to lon and lat
#     .rename(
#         {
#             "latitude": "lat",
#             "longitude": "lon",
#         }
#     )
# )
# ds_forecast

## Temperature

In [22]:
surface_temp = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param=["2t", "2d"],
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()
surface_temp

<xarray.Dataset> Size: 2GB
Dimensions:    (number: 50, step: 2, latitude: 721, longitude: 1440)
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    2d         (number, step, latitude, longitude) float64 831MB ...
    2t         (number, step, latitude, longitude) float64 831MB ...
Attributes:
    class:        od
    stream:       enfo
    levtype:      sfc
    type:         pf
    expver:       0001
    date:         20250926
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

In [26]:
soil_temp = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param="sot", # soil temperature
    levtype="sol",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray().sel(level=4)
soil_temp

<xarray.Dataset> Size: 831MB
Dimensions:    (number: 50, step: 2, latitude: 721, longitude: 1440)
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
    level      int64 8B 4
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    sot        (number, step, latitude, longitude) float64 831MB ...
Attributes:
    param:        sot
    paramId:      260360
    class:        od
    stream:       enfo
    levtype:      sol
    type:         pf
    expver:       0001
    date:         20250926
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

## Runoff

In [27]:
runoff = ekd.from_source(
    date=forecast_date,
    time=forecast_time,
    param="ro",
    levtype="sfc",
    # step=list(range(0, 144+3, 3)) + list(range(150, 360+6, 6)),
    step=steps,
    **model_config
).to_xarray()
runoff

<xarray.Dataset> Size: 831MB
Dimensions:    (number: 50, step: 2, latitude: 721, longitude: 1440)
Coordinates:
  * number     (number) int64 400B 1 2 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49 50
  * step       (step) timedelta64[ns] 16B 18:00:00 1 days
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    ro         (number, step, latitude, longitude) float64 831MB ...
Attributes:
    param:        ro
    paramId:      205
    class:        od
    stream:       enfo
    levtype:      sfc
    type:         pf
    expver:       0001
    date:         20250926
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF